In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
batch_size =32
num_classes =10
epochs= 10

img_rows, img_cols =28,28

In [5]:
fashion_mnist = keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()

In [6]:
x_train = x_train.reshape(x_train.shape[0],img_rows, img_cols,1)
x_test = x_test.reshape(x_test.shape[0],img_rows, img_cols,1)
input_shape = (img_rows, img_cols,1)

In [7]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255

In [8]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [9]:
# preprocess function
# normalize x
def preprocess(x, y):
  x = x/255
  x = tf.expand_dims(x, 3)
  return x, y

In [10]:
# make trainset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.map(preprocess)

test_dataset  = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.map(preprocess)

In [11]:
num_class =10

with tf.device('/device:GPU:0'):
  # model layer
  model = keras.models.Sequential([
              keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)), ## 입력배열 28,28,1
              keras.layers.MaxPool2D(pool_size=(2, 2)),
              keras.layers.Dropout(0.25),
              keras.layers.Flatten(),
              keras.layers.Dense(128, activation='relu'),
              keras.layers.Dropout(0.5),
              keras.layers.Dense(num_class, activation='softmax') ## 출력 class = num_class
  ])

  # optimizer, loss, 성능평가 척도 설정
  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics='accuracy'
  )

In [12]:
## Data augmentation (회전, 이동, 비틀기, 확대)

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest')

In [13]:
train_data_flow = datagen.flow(x_train, y_train, batch_size=10) #x_train, y_train

In [14]:
len(train_data_flow)

6000

In [15]:
with tf.device('/device:GPU:0'):
  hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), validation_data=(x_test, y_test),epochs=10)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1875/1875 [==============================] - 58s 31ms/step - loss: 1.2365 - accuracy: 0.5444 - val_loss: 0.7417 - val_accuracy: 0.7152
Epoch 2/10
1875/1875 [==============================] - 57s 30ms/step - loss: 1.0007 - accuracy: 0.6261 - val_loss: 0.7243 - val_accuracy: 0.7196
Epoch 3/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.9389 - accuracy: 0.6464 - val_loss: 0.6613 - val_accuracy: 0.7331
Epoch 4/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.9048 - accuracy: 0.6602 - val_loss: 0.6549 - val_accuracy: 0.7341
Epoch 5/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.8738 - accuracy: 0.6710 - val_loss: 0.6278 - val_accuracy: 0.7446
Epoch 6/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.8562 - accuracy: 0.6755 - val_loss: 0.6550 - val_accuracy: 0.7431
Epoch 7/10
1875/1875 [==============================] - 56s 30ms/step - loss: 0.8509 - accuracy: 0.6813 - val_loss: 0.6934 -